In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from torchvision import models
from sklearn.metrics import roc_auc_score

# ==== 1. SETTING UP GPU ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==== 2. PARAMETERS ====
data_dir = "/kaggle/input/pneumonia/pneumonia-kaggle/train"  # Directory containing training images
batch_size = 32  # Number of images per batch
epochs = 7  # Number of training epochs
learning_rate = 1e-3  # Learning rate for the optimizer
train_ratio = 0.8  # 80% for training, 20% for validation

# ==== 3. DATA TRANSFORMATIONS ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

# ==== 4. LOADING DATA ====
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(train_ratio * len(dataset))  # Compute training set size
val_size = len(dataset) - train_size  # Compute validation set size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])  # Split dataset

# Balancing class distribution in the training set
train_labels = [dataset.targets[i] for i in train_dataset.indices]  # Get labels of training samples
class_weights = [1 / train_labels.count(0), 1 / train_labels.count(1)]  # Compute class weights
sample_weights = [class_weights[label] for label in train_labels]  # Assign weights to each sample
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)  # Create a weighted sampler

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# ==== 5. LOADING ResNet-50 ====
model = models.resnet50(pretrained=True)  # Load pre-trained ResNet-50 model
model.fc = nn.Linear(model.fc.in_features, 1)  # Modify the classifier for binary classification (1 output)
model = model.to(device)  # Move model to GPU or CPU

# ==== 6. OPTIMIZER AND LOSS FUNCTION ====
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)  # AdamW optimizer
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)  # Learning rate scheduler

# ==== 7. TRAINING LOOP ====
best_auc = 0.0  # Variable to track the best validation AUC
for epoch in range(epochs):
    if epoch>4:
        learning_rate = 1e-4
    model.train()  # Set model to training mode
    train_loss = 0.0  # Track training loss
    all_preds, all_labels = [], []  # Store predictions and labels
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Move data to GPU/CPU
        optimizer.zero_grad()  # Zero gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        train_loss += loss.item()  # Accumulate loss
        all_preds.extend(torch.sigmoid(outputs).cpu().detach().numpy())  # Store predictions
        all_labels.extend(labels.cpu().detach().numpy())  # Store true labels
    
    train_auc = roc_auc_score(all_labels, all_preds)  # Compute AUC for training set
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0  # Track validation loss
    val_preds, val_labels = [], []  # Store validation predictions and labels
    with torch.no_grad():  # Disable gradient calculation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Move data to GPU/CPU
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            val_loss += loss.item()  # Accumulate loss
            val_preds.extend(torch.sigmoid(outputs).cpu().detach().numpy())  # Store predictions
            val_labels.extend(labels.cpu().detach().numpy())  # Store true labels
    
    val_auc = roc_auc_score(val_labels, val_preds)  # Compute AUC for validation set
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss/len(train_loader):.4f} - Train AUC: {train_auc:.4f} - Val Loss: {val_loss/len(val_loader):.4f} - Val AUC: {val_auc:.4f}")
    scheduler.step()  # Adjust learning rate
    
    # Save the best model based on validation AUC
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), "resnet50-imagenet.pth")  # Save model weights
        print("Best model saved!")

print("Training complete. Best Validation ROC AUC:", best_auc)


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/7 - Train Loss: 0.1356 - Train AUC: 0.9881 - Val Loss: 0.1214 - Val AUC: 0.9970
Best model saved!
Epoch 2/7 - Train Loss: 0.0736 - Train AUC: 0.9962 - Val Loss: 0.0624 - Val AUC: 0.9971
Best model saved!
Epoch 3/7 - Train Loss: 0.0589 - Train AUC: 0.9978 - Val Loss: 0.1103 - Val AUC: 0.9948
Epoch 4/7 - Train Loss: 0.0282 - Train AUC: 0.9996 - Val Loss: 0.0400 - Val AUC: 0.9990
Best model saved!
Epoch 5/7 - Train Loss: 0.0169 - Train AUC: 0.9999 - Val Loss: 0.0320 - Val AUC: 0.9992
Best model saved!
Epoch 6/7 - Train Loss: 0.0082 - Train AUC: 1.0000 - Val Loss: 0.0388 - Val AUC: 0.9990
Epoch 7/7 - Train Loss: 0.0111 - Train AUC: 0.9997 - Val Loss: 0.0411 - Val AUC: 0.9990
Training complete. Best Validation ROC AUC: 0.9992367975279686


In [ ]:
import torch
import os
import pandas as pd
from PIL import Image
from torchvision import transforms

# ==== 8. UPLOAD THE BEST MODEL ====
model.load_state_dict(torch.load("/kaggle/working/resnet50-imagenet.pth"))  # Load the best ResNet-50 model
model.eval()

# ==== 9. CREATE SUBMISSION ====
test_dir = "/kaggle/input/pneumonia/pneumonia-kaggle/test"
test_images = sorted(os.listdir(test_dir))
submission = []

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()  

    submission.append([img_name, prediction])

df = pd.DataFrame(submission, columns=["Id", "Category"])
df.to_csv("/kaggle/working/resnet50-imagenet.csv", index=False)

print("Submission file saved!")


<ipython-input-4-75b402100bfb>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/resnet50-imagenet.pth"))  # Load the best R

Submission file saved!
